In [1]:
import pandas as pd
import requests
import os
import datetime
import time
import random
from dotenv import load_dotenv

## Validar o número de contrato e extrair informações do objeto do contrato a partir da API

### Carregar o dataframe final tratado

In [2]:
df_final = pd.read_csv("../data/terceirizados/arquivos_tratados/consolidado_validacao.csv", dtype="str")

# Padronizar os dados antes de chamar a API
df_final['nr_contrato'] = df_final['nr_contrato'].astype(str).str.strip()
df_final['cd_orgao_siafi'] = df_final['cd_orgao_siafi'].astype(str).str.strip()
df_final['cnpj_formatado'] = df_final['cnpj_formatado'].astype(str).str.strip()

In [3]:
df_final.head()

,nr_contrato,cd_orgao_siafi,nm_razao_social,sg_orgao_sup_tabela_ug,cd_ug_gestora,nm_ug_tabela_ug,sg_ug_gestora,nr_cnpj,nm_unidade_prestacao,sg_orgao,nm_orgao,cd_orgao_siape,cnpj_formatado
0,000002019,26273,SULCLEAN SERVICOS LTDA.,MIN.EDUC,154042,UNIVERIDADE FEDERAL DO RIO GRANDE - FURG,FURG,6205427000102,CIDEC / REITORIA,FURG,UNIVERSIDADE FEDERAL DO RIO GRANDE - FURG,26273,06.205.427/0001-02
1,000002023,20411,XYZ LTDA,MINC,343029,SUPERINTENDENCIA DO IPHAN NO DISTRITO FEDERAL,IPHAN-DF,0,IPHAN - DF,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204,00.000.000/0000-00
2,000012011,20411,POLO SEGURANCA ESPECIALIZADA LTDA.,MINIST.,343002,SUPERINTENDENCIA DO IPHAN NO PARA,IPHAN-PA,02650833000123,PREDIO SEDE/PA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204,02.650.833/0001-23
3,000012012,20411,ARE SERVICOS LTDA,MINIST.,343003,IPHAN - SUPERINTENDENCIA DO MARANHAO,IPHAN-MA,11873594000161,SUPERINTENDENCIA,IPHAN,INSTITUTO DO PATRIMONIO HIST. E ART. NACIONAL,24204,11.873.594/0001-61
4,000012012,36212,RM SERVILOS EMPRESARIAIS LTDA-ME,MINISTER,253029,COORDENACAO ESTADUAL DE VIGILANCIA SANITARIA D...,CVSPAF-AC/ANVISA,09646758000176,CVPAF/AC-SEDE,ANVS,AGENCIA NACIONAL DE VIGILANCIA SANITARIA,36207,09.646.758/0001-76


### Configurar a API para requisição e validação

In [2]:
load_dotenv()
API_KEY = os.getenv("API_PORTAL_TRANSPARENCIA")

In [3]:
base_url = "https://api.portaldatransparencia.gov.br/api-de-dados/contratos"
headers = {
    "chave-api-dados": API_KEY,
    "Accept": "application/json",
}

In [5]:
# Criar sessão persistente para melhorar a performance
session = requests.Session()
session.headers.update(headers)

In [6]:
# Limites de requisições por minuto conforme a API
max_requisicoes_dia = 90   # 06:00 - 23:59
max_requisicoes_madrugada = 300  # 00:00 - 05:59
historico_requisicoes = []  # Armazena timestamps das requisições

In [7]:
# Diretórios
caminho_checkpoint = "../data/terceirizados/arquivos_para_validacao/"
caminho_saida_final = "../data/terceirizados/arquivos_tratados/contratos_validados_completo.csv"

### Carregar arquivos já validados para não repetir o processo

In [8]:
arquivos_parciais = [os.path.join(caminho_checkpoint, f) for f in os.listdir(caminho_checkpoint) if f.endswith(".csv")]
df_parciais = [pd.read_csv(arquivo, dtype=str) for arquivo in arquivos_parciais] if arquivos_parciais else []

In [9]:
# Consolidar arquivos já validados
df_validado_parcial = pd.concat(df_parciais, ignore_index=True) if df_parciais else pd.DataFrame()

if not df_validado_parcial.empty:
    contratos_validados = df_validado_parcial[['nr_contrato', 'cd_orgao_siafi']]
    df_faltantes = df_final.merge(contratos_validados, on=['nr_contrato', 'cd_orgao_siafi'], how='left', indicator=True)
    df_faltantes = df_faltantes[df_faltantes['_merge'] == 'left_only'].drop(columns=['_merge'])
else:
    df_faltantes = df_final.copy()

print(f"📌 Contratos pendentes para validação: {len(df_faltantes)}")
df_faltantes.to_csv("../data/terceirizados/arquivos_para_validacao/contratos_pendentes.csv", index=False)

📌 Contratos pendentes para validação: 22055


In [10]:
def respeitar_limite_requisicoes():
    """Garante que as requisições respeitam os limites da API."""
    global historico_requisicoes

    agora = datetime.datetime.now()
    limite_requisicoes = max_requisicoes_dia if 6 <= agora.hour <= 23 else max_requisicoes_madrugada

    # Remover registros mais antigos que 60 segundos
    historico_requisicoes = [t for t in historico_requisicoes if (agora - t).seconds < 60]

    if len(historico_requisicoes) >= limite_requisicoes:
        tempo_espera = 60 - (agora - historico_requisicoes[0]).seconds
        print(f"⏳ Limite de {limite_requisicoes} requisições atingido. Aguardando {tempo_espera} segundos...")
        time.sleep(tempo_espera)

    historico_requisicoes.append(datetime.datetime.now())

### Função para buscar os dados na API

In [11]:
# Variável global para armazenar ocorrências
erros_contratos = []

def buscar_dados_contrato(row):
    numero_contrato = row['nr_contrato']
    orgao_siafi = row['cd_orgao_siafi']
    cnpj_formatado = row['cnpj_formatado']

    tentativas_maximas = 3

    for tentativa in range(tentativas_maximas):
        respeitar_limite_requisicoes()
        time.sleep(random.uniform(0.5, 1.5))
        
        try:
            url = f"{base_url}?numero={numero_contrato}&codigoOrgao={orgao_siafi}"
            response = session.get(url, timeout=10)

            if response.status_code != 200:
                continue  

            try:
                data = response.json()
            except ValueError:
                continue  

            if not data:
                erros_contratos.append(f"{numero_contrato} - Não encontrado")
                return {"status_validacao": "Não encontrado"}

            if len(data) == 1:
                contrato = data[0]
            else:
                contrato = next(
                    (contrato for contrato in data 
                     if cnpj_formatado == contrato['fornecedor'].get('cnpjFormatado', '').strip()),
                    None
                )

                if not contrato:
                    erros_contratos.append(f"{numero_contrato} - CNPJ não corresponde")
                    return {"status_validacao": "CNPJ não corresponde"}

            return {
                "id_contrato": contrato.get("id", ""),
                "objeto_contrato": contrato.get("objeto", ""),
                "objeto_compra": contrato.get("compra", {}).get("objeto", ""),
                "dataAssinatura": contrato.get("dataAssinatura", ""),
                "dataPublicacaoDOU": contrato.get("dataPublicacaoDOU", ""),
                "dataInicioVigencia": contrato.get("dataInicioVigencia", ""),
                "dataFimVigencia": contrato.get("dataFimVigencia", ""),
                "tipo_fornecedor": contrato.get("fornecedor", {}).get("tipo", ""),
                "valorInicialCompra": contrato.get("valorInicialCompra", ""),
                "valorFinalCompra": contrato.get("valorFinalCompra", "")
            }

        except requests.exceptions.RequestException:
            continue  

    erros_contratos.append(f"{numero_contrato} - Erro na API")
    return {"status_validacao": "Erro na API"}

### Processar os contratos em lotes

In [12]:
tamanho_lote = 1000  # Processar 1.000 registros por vez
df_corrigido = pd.DataFrame()

for i in range(0, len(df_faltantes), tamanho_lote):
    print(f"🔍 Processando lote {i // tamanho_lote + 1} de {len(df_faltantes) // tamanho_lote + 1}...")

    df_lote = df_faltantes.iloc[i:i + tamanho_lote].copy()
    dados_api = df_lote.apply(buscar_dados_contrato, axis=1)
    df_api = pd.DataFrame(dados_api.tolist())

    df_lote = pd.concat([df_lote.reset_index(drop=True), df_api.reset_index(drop=True)], axis=1)
    df_lote.to_csv(os.path.join(caminho_checkpoint, f"contratos_validacao_parcial_{i}.csv"), index=False)
    df_corrigido = pd.concat([df_corrigido, df_lote], ignore_index=True)

    respeitar_limite_requisicoes()
    time.sleep(random.uniform(1, 3))  # Reduzi para otimizar o tempo sem sobrecarregar a API

# Salvar relatório de erros
if erros_contratos:
    with open("erros_validacao.txt", "w") as f:
        for erro in erros_contratos:
            f.write(f"{erro}\n")
    print(f"📄 Log de erros salvo com {len(erros_contratos)} inconsistências.")

print(f"✅ Processo concluído! Total de registros processados: {len(df_corrigido)}")

🔍 Processando lote 1 de 23...
🔍 Processando lote 2 de 23...
🔍 Processando lote 3 de 23...
🔍 Processando lote 4 de 23...
🔍 Processando lote 5 de 23...
🔍 Processando lote 6 de 23...
🔍 Processando lote 7 de 23...
🔍 Processando lote 8 de 23...
🔍 Processando lote 9 de 23...
🔍 Processando lote 10 de 23...
🔍 Processando lote 11 de 23...
🔍 Processando lote 12 de 23...
🔍 Processando lote 13 de 23...
🔍 Processando lote 14 de 23...
🔍 Processando lote 15 de 23...
🔍 Processando lote 16 de 23...
🔍 Processando lote 17 de 23...
🔍 Processando lote 18 de 23...
🔍 Processando lote 19 de 23...
🔍 Processando lote 20 de 23...
🔍 Processando lote 21 de 23...
🔍 Processando lote 22 de 23...
🔍 Processando lote 23 de 23...
📄 Log de erros salvo com 21451 inconsistências.
✅ Processo concluído! Total de registros processados: 22055


## Consolidar os arquivos com os contratos validados para verificar as inconsistências

In [13]:
df_validado_final = pd.concat([df_validado_parcial, df_corrigido], ignore_index=True)
df_validado_final.to_csv(caminho_saida_final, index=False, float_format="%.2f")

print(f"🚀 Arquivo final consolidado salvo em: {caminho_saida_final}")

🚀 Arquivo final consolidado salvo em: ../data/terceirizados/arquivos_tratados/contratos_validados_completo.csv
